<a href="https://colab.research.google.com/github/KoMurase/Learning_my_interest/blob/master/%E3%83%AC%E3%82%B3%E3%83%A1%E3%83%B3%E3%83%89%E3%82%B7%E3%82%B9%E3%83%86%E3%83%A0_NeuralMatrixFactorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.4.0-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision
!pip3 install tqdm

     |████████████████████████████████| 484.0MB 1.9MB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.4.0 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.4.0 which is incompatible.
  Found existing installation: torch 1.3.1
    Uninstalling torch-1.3.1:
      Successfully uninstalled torch-1.3.1
     |████████████████████████████████| 734.6MB 9.3kB/s 
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0


In [0]:
import torch
torch.tensor([1,2,3]).to("cuda:0")
from torch import nn, optim
from torch.utils.data import (Dataset, 
                              DataLoader,
                              TensorDataset)
import tqdm

In [3]:
!wget http://files.grouplens.org/datasets/movielens/ml-20m.zip
!unzip ml-20m.zip

--2019-12-29 17:49:29--  http://files.grouplens.org/datasets/movielens/ml-20m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 198702078 (189M) [application/zip]
Saving to: ‘ml-20m.zip’

ml-20m.zip          100%[===================>] 189.50M  55.8MB/s    in 3.7s    

2019-12-29 17:49:33 (50.9 MB/s) - ‘ml-20m.zip’ saved [198702078/198702078]

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         


In [0]:
class NeuralMetricsFactrization(nn.Module):
  def __init__(self, max_user, max_item, user_k=10, item_k=10, hidden_dim=50):
    super().__init__()
    self.user_emb = nn.Embedding(max_user, user_k, 0)
    self.item_emb = nn.Embedding(max_item, item_k, 0)

    self.mlp = nn.Sequential(
        max_Linear(user_k + item_k, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim), 
        nn.Linear(hidden_dim, 1)
    )
  
  def forward(self, x):
    user_idx = x[:, 0]
    item_idx = x[:, 1]
    user_feature = self.user_emb(user_idx)
    item_feature = self.item_emb(item_idx)
    #ユーザーの特徴量と商品特徴量をまとめて一つのベクトルにする
    out = torch.cat([user_feature, item_feature], 1)
    #まとめて特徴量ベクトルをMLPに入れる
    out = self.mlp(out)
    out = nn.functional.sigmoid(out) * 5 
    return out.squeeze()


    ###

区切られたジャンルをBoWに変換

In [0]:
import csv  
from sklearn.feature_extraction.text import CountVectorizer 

#csv.DictReaderを展開してcsvファイルを読み込み
#必要な部分だけ抽出する
with open("./ml-20m/movies.csv") as fp:
  reader = csv.DictReader(fp)
  def parse(d):
    movieId = int(d["movieId"])
    genres = d["genres"]
    return movieId, genres
  data = [parse(d) for d in reader] 

In [6]:
data[0:10]

[(1, 'Adventure|Animation|Children|Comedy|Fantasy'),
 (2, 'Adventure|Children|Fantasy'),
 (3, 'Comedy|Romance'),
 (4, 'Comedy|Drama|Romance'),
 (5, 'Comedy'),
 (6, 'Action|Crime|Thriller'),
 (7, 'Comedy|Romance'),
 (8, 'Adventure|Children'),
 (9, 'Action'),
 (10, 'Action|Adventure|Thriller')]

In [0]:
movieIds = [x[0] for x in data]
genres = [x[1] for x in data]

In [8]:
movieIds[0:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
genres[0:10]

['Adventure|Animation|Children|Comedy|Fantasy',
 'Adventure|Children|Fantasy',
 'Comedy|Romance',
 'Comedy|Drama|Romance',
 'Comedy',
 'Action|Crime|Thriller',
 'Comedy|Romance',
 'Adventure|Children',
 'Action',
 'Action|Adventure|Thriller']

In [0]:
#データに合わせてCountVectorizerを訓練する
cv = CountVectorizer(dtype="f4").fit(genres)
num_genres = len(cv.get_feature_names())

#keyがmovieIdでvalueがBoWのTensorのdictを作る
it = cv.transform(genres).toarray()
it = (torch.tensor(g, dtype=torch.float32) for g in it)
genre_dict = dict(zip(movieIds, it))

In [11]:
cv

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype='f4', encoding='utf-8', input='content', lowercase=True,
                max_df=1.0, max_features=None, min_df=1, ngram_range=(1, 1),
                preprocessor=None, stop_words=None, strip_accents=None,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
                vocabulary=None)

In [12]:
num_genres

24

In [13]:
it

<generator object <genexpr> at 0x7f8057159e60>

In [0]:
def first(xs):
  it = iter(xs)
  return next(it)

class MovieLensDataset(Dataset):
  def __init__(self, x, y, genres):
    #assert 条件式, 条件式がFalseの場合に出力するメッセージ
    #条件式がFalseの場合, AssertionError の例外が発生します。
    #条件式が True の場合は何も起こりません。
    assert len(x) == len(y)
    self.x = x
    self.y = y 
    self.genres = genres 
    
    #ジャンル辞書にないmovieIdの時のダミーデータ
    self.null_genre = torch.zeros_like(
        first(genres.values())
    )
  
  def __len__(self):
    return len(self.x)
  
  def __getitem__(self, idx):
    x = self.x[idx]
    y = self.y[idx]
    # x = (userId , movieId)
    movieId = x[1]
    g = self.genres.get(movieId, self.null_genre)
    return x, y, g

In [0]:
import pandas as pd 
from sklearn import model_selection 

df = pd.read_csv("/content/ml-20m/ratings.csv")

X = df[["userId", "movieId"]].values
Y = df[["rating"]].values

train_X, test_X, train_Y, test_Y = model_selection.train_test_split(X,Y,test_size=0.1)

In [0]:
train_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32),
    genre_dict
)
test_dataset = MovieLensDataset(
    torch.tensor(train_X, dtype=torch.int64),
    torch.tensor(train_Y, dtype=torch.float32),
    genre_dict
)
train_loader = DataLoader(
    train_dataset, 
    batch_size = 1024, 
    shuffle = True, 
    num_workers=4
)
test_size = DataLoader(
    test_dataset,
    batch_size=1024,
    num_workers=4
)

ネットワークモデルの作成

In [0]:
class NeuralMatrixFactorization2(nn.Module):
  def __init__(self, max_user, max_item, num_genres, user_k=10, item_k=10, hidden_dim=50):
    super().__init__()
    self.user_emb = nn.Embedding(max_user, user_k, 0)
    self.item_emb = nn.Embedding(max_item, item_k, 0)
    self.mlp = nn.Sequential(
        # num_genres分だけ次元が増える
        nn.Linear(user_k + item_k + num_genres, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, 1)
    )
  
  def forword(self, x, g):
    user_idx = x[:, 0]
    item_idx = x[:, 1]
    user_feature = self.user_emb(user_idx)
    item_feature = self.item_emb(item_idx)
    #ジャンルのBoWをcatで特徴ベクトルに結合する
    out = torch.cat([user_feature, item_feature, g], 1)
    out = self.mlp(out)
    out = nn.functional.sigmoid(out) * 5
    return out.squeeze()

In [0]:
 def eval_net(net, loader, score_fn=nn.functional.l1_loss, device="cpu"):
  ys = []
  ypreds = []
  #loaderはジャンルのBoWも返す
  for x, y, g in loader:
    x = x.to(device)
    g = g.to(device)
    ys.append(y)
    # userld, movield以外にジャンルの
    # BoWもネットワーク関数に渡す
    with torch.no_grad():
      ypred = net(x, g).to("cpu")
    ypreds.append(ypred)
  score = score_fn(torch.cat(ys).sequeeze(), torch.cat(ypreds))
  return score

In [0]:
max_user, max_item = X.max(0)
max_user = int(max_user)
max_item = int(max_item)

In [39]:
#訓練部分の作成
net = NeuralMatrixFactorization2(
    max_user+1, max_item+1, num_genres
)
opt = optim.Adam(net.parameters(), lr=0.01)
loss_f = nn.MSELoss()

net.to("cuda:0")
for epoch in range(5):
  loss_log = []
  net.train()
  for x, y, g in tqdm.tqdm(train_loader):
    x = x.to("cuda:0")
    y = y.to("cuda:0")
    g = g.to("cuda:0")
    o = net(x, g)
    loss = loss_f(o, y.view(-1))
    net.zero_grad()
    loss.backward()
    opt.step()
    loss_log.append(loss.item())
  net.eval()
  test_score = eval_net(net, test_loader, device="cuda:0")
  print(epoch, mean(loss_log), test_score.item(), flush=True)

  0%|          | 0/17579 [00:00<?, ?it/s]


NotImplementedError: ignored

In [0]:
# 指定した位置だけ1で残りが0のTensorを返す補助関数
def make_genre_vector(i, max_len):
  g = torch.zeros(max_len)
  g[i] = 1
  return g 

query_genres = [make_genre_vector(i, num_genres) for i in range(num_genres)]
query_genres = torch.stack(query_genres, 1)

# num_genres分だけuser_Id=100, movieId=0のTensorを作って結合する
query = torch.stack([
                     torch.empty(num_genres, dtype=torch.int64).fill_(100),
                     torch.empty(num_genres, dtype=torch.int64).fill_(0)
], 1)

#GPUに転送
query_genres = query_genres.to("cuda:0")
query = query.to("cuda:0")

